# Setup

### Imports

In [7]:
import sys

sys.path.append("/Users/64926/projects/tarsila/braz-speech-pipeline/")

import pandas as pd
from Levenshtein import distance
from pathlib import Path
import os
from tqdm import tqdm

from src.clients.database import Database
from src.clients.scp_transfer import FileTransfer, ContentType

### Queries

In [8]:
def get_segments_by_audio_id(audio_id):
    with Database() as db: 
        return db._run_query(f"""
        SELECT *
        FROM Dataset
        WHERE audio_id = {audio_id}
        """)


def get_all_audios():
    with Database() as db: 
        return db._run_query(f"""
        SELECT *
        FROM Audio
        WHERE corpus_id is not null
        """)

### Constants

In [9]:
BASE_DATASET_REMOTRE_DIR = Path("~/BrazSpeechData/static/Dataset/data/")

# Analysis

For the database sanity check, we want to analize three problems:

1. Duplicated entries
2. Missing segments
3. Stuck audios in annotators that left the project
4. Wrong start_time, end_time and durations (missing offset)

### 1. Duplicated entries

There are different scenarios where we can consider an entry to be duplicated:

- **Exact duplication:** where something like `"PC_MA_HV242_Vicente Sacchi" == "PC_MA_HV242_Vicente Sacchi"` happens.
- **Duplication without spaces:** it's also possible to have something like `"PC_MA_HV242_Vicente Sacchi" == "PC_MA_HV242_Vicente_Sacchi"`
- **Duplications of the code:** finally, we have cases where the audio only contains the code and it's duplicated, like `"PC_MA_HV242_Vicente Sacchi" == "PC_MA_HV242"`

Considering these three cases, if we analize only the last one, we'll be good!

In [10]:
with Database() as db:
    duplicated_audios = db._run_query(
        """SELECT a.*
FROM Audio a
WHERE EXISTS (
    SELECT 1
    FROM (
        SELECT SUBSTRING_INDEX(name, '_', 3) AS code
        FROM Audio
        GROUP BY code
        HAVING COUNT(*) > 1
    ) AS DuplicatedAudios
    WHERE a.name LIKE CONCAT(DuplicatedAudios.code, '%')
)
ORDER BY name ASC;"""
    )
duplicated_audios

,id,created_at,name,num_speakers,json_metadata,duration,corpus_id,finished,responsible_user_id,to_review,reviewer_user_id,last_segment_num_reviewed,error_flag
0,500,2023-09-11 06:13:00,PC_MA_HV203_Samuel Gonçalves,None,"{""race"": null, ""title"": ""\""Quero contribuir co...",5384.08,1,1.0,128,None,None,None,NaN
1,534,2023-09-11 22:30:19,PC_MA_HV203_Samuel Gonçalves,None,"{""race"": null, ""title"": ""\""Quero contribuir co...",5384.08,1,NaN,128,None,None,None,1.0
2,502,2023-09-11 07:32:07,PC_MA_HV205_Cássia Navas Alves de Castro,None,"{""race"": null, ""title"": ""A dança através da pe...",5709.65,1,1.0,128,None,None,None,NaN
3,533,2023-09-11 22:09:53,PC_MA_HV205_Cássia Navas Alves de Castro,None,"{""race"": null, ""title"": ""A dança através da pe...",5709.65,1,1.0,127,None,None,None,NaN
4,525,2023-09-11 19:42:47,PC_MA_HV206_Guiomar Rocha Alvares,None,"{""race"": null, ""title"": ""Memórias de uma cente...",5766.00,1,1.0,128,None,None,None,NaN
5,535,2023-09-11 22:48:32,PC_MA_HV206_Guiomar Rocha Alvares,None,"{""race"": null, ""title"": ""Memórias de uma cente...",5766.00,1,NaN,127,None,None,None,1.0


In [11]:
# Now we can delete the duplicated audios that were not started yet (i.e. responsible user id is null)

assert isinstance(
    duplicated_audios, pd.DataFrame
), "duplicated_audios is not a dataframe"

ids_to_delete = duplicated_audios[
    duplicated_audios["responsible_user_id"].isna()
].id.to_list()

print (f"Deleting {len(ids_to_delete)} audios: {ids_to_delete}")

if len(ids_to_delete) > 0:
    delete_segments_query = f"""
    DELETE
    FROM Dataset
    WHERE audio_id in {tuple(ids_to_delete)}
    """

    delete_audios_query = f"""
    DELETE
    FROM Audio
    WHERE id in {tuple(ids_to_delete)}
    """

    with Database() as db:
        db._run_query(delete_segments_query)
        db._run_query(delete_audios_query)

Deleting 0 audios: []


In [12]:

assert isinstance(
    duplicated_audios, pd.DataFrame
), "duplicated_audios is not a dataframe"

# Step 1: Identify duplicated audio names
duplicated_audio_names = duplicated_audios["name"].unique()

# Initialize an empty list to store the results
difference_results = []

# Iterate over each duplicated audio name
for audio_name in duplicated_audio_names:
    # Get the IDs of the duplicated audio entries
    audio_ids = duplicated_audios[duplicated_audios["name"] == audio_name][
        "id"
    ].tolist()

    # Fetch the corresponding segments for each audio ID
    segments1 = get_segments_by_audio_id(audio_ids[0])
    segments1_texts = segments1["text"]
    segments2 = get_segments_by_audio_id(audio_ids[1])
    segments2_texts = segments2["text"]

    # Calculate the Levenshtein distance for each pair of segments
    for text1, text2 in zip(segments1_texts, segments2_texts):
        if text1 is not None and text2 is not None:
            lev_distance = distance(text1, text2)
        else:
            lev_distance = None
        difference_results.append(
            (audio_name, audio_ids[0], audio_ids[1], text1, text2, lev_distance)
        )

# Convert the results to a DataFrame
difference_df = pd.DataFrame(
    difference_results, columns=["audio_name", "audio_id1", "audio_id2", "text_1", "text2", "lev_distance"]
)

difference_df

,audio_name,audio_id1,audio_id2,text_1,text2,lev_distance
0,PC_MA_HV203_Samuel Gonçalves,500,534,###,###,0.0
1,PC_MA_HV203_Samuel Gonçalves,500,534,###,###,0.0
2,PC_MA_HV203_Samuel Gonçalves,500,534,###,###,0.0
3,PC_MA_HV203_Samuel Gonçalves,500,534,###,###,0.0
4,PC_MA_HV203_Samuel Gonçalves,500,534,"E você morava com seus pais, com seus avós, co...","E você morava com seus pais, com seus avós, co...",7.0
...,...,...,...,...,...,...
3508,PC_MA_HV206_Guiomar Rocha Alvares,525,535,###,None,NaN
3509,PC_MA_HV206_Guiomar Rocha Alvares,525,535,###,None,NaN
3510,PC_MA_HV206_Guiomar Rocha Alvares,525,535,"Está bem, bem.",None,NaN
3511,PC_MA_HV206_Guiomar Rocha Alvares,525,535,Eu só tenho prazer.,None,NaN


In [13]:
difference_df[["text_1", "text2", "lev_distance"]].sort_values(by="lev_distance", ascending=False).head(200)

,text_1,text2,lev_distance
1753,###,"E tem as trovas acadêmicas, e tem o Centro Aca...",463.0
1697,###,"debutante no mundo, aí eu fiquei um ano meio z...",442.0
2243,###,"todas as estruturas no Centro Cultural, que, v...",390.0
1206,###,"Então, eh e meu, não sem um, uma certa, na épo...",279.0
2148,###,"Bom, estou no Departamento do Patrimônio Histó...",275.0
...,...,...,...
2823,"Mas, na Secretaria da Cultura, nunca entendi.....","Mas, na Secretaria de Agricultura, nunca enten...",19.0
2084,"Então tem esse momento, dança, aham, formação,...","Então tem esse momento, dança, né, formação, e...",18.0
2613,E aquelas ( ) vizinhas.,E aquelas vizinhas estava assim.,18.0
1836,"Diferentemente disso, imperou o bom senso e al...","Diferentemente disso, imperou o bom senso e al...",18.0


In [14]:
difference_df.to_csv("duplicated_audios.csv")

In [15]:
difference_df[difference_df.lev_distance > 0]

,audio_name,audio_id1,audio_id2,text_1,text2,lev_distance
4,PC_MA_HV203_Samuel Gonçalves,500,534,"E você morava com seus pais, com seus avós, co...","E você morava com seus pais, com seus avós, co...",7.0
5,PC_MA_HV203_Samuel Gonçalves,500,534,Isso. É.,Isso.,3.0
6,PC_MA_HV203_Samuel Gonçalves,500,534,É...Mo( ) morei muito tempo eh no no terreno d...,Eh... Eu morei muito tempo no terreno do meu a...,15.0
7,PC_MA_HV203_Samuel Gonçalves,500,534,"Ih... logo depois que eu nasci, um tempo depoi...","E... logo depois que eu nasci, um tempo depois...",8.0
11,PC_MA_HV203_Samuel Gonçalves,500,534,"Meu avô paterno já faleceu, né, um pouco antes...",Meu avô paterno já faleceu um pouco antes de e...,5.0
...,...,...,...,...,...,...
2846,PC_MA_HV206_Guiomar Rocha Alvares,525,535,"E eu para ser professora naquele tempo, eu era...","E eu para ser professora naquele tempo, eu era...",2.0
2848,PC_MA_HV206_Guiomar Rocha Alvares,525,535,Você saía de sua casa e ia morar numa fazenda ...,Você saía de sua casa e ia morar numa fazenda ...,15.0
2849,PC_MA_HV206_Guiomar Rocha Alvares,525,535,Muito poucos eram corretos.,"Muito poucos eram, ah, corretos. e",7.0
2850,PC_MA_HV206_Guiomar Rocha Alvares,525,535,"E, de fato, eu tinha medo, porque essa históri...","E, de fato, eu tinha medo, porque eh, essa his...",10.0


### 2. Missing segments

In this analysis, we need to understand if the data in the database is the same than the data on the raw files, to make sure we're not missing any segment.

In [16]:
with Database() as db:
    audios_on_db = get_all_audios()

In [17]:
with FileTransfer() as ft:
    nurc_files = ft.list_directory_contents(
        BASE_DATASET_REMOTRE_DIR / "nurc_sp",
        content_type=ContentType.FILES,
        recursive=True,
    )

    mupe_files = ft.list_directory_contents(
        BASE_DATASET_REMOTRE_DIR / "mupe",
        content_type=ContentType.FILES,
        recursive=True,
    )

audios_on_folders = {}


def process_path(acc, corpus_name, path):
    path = path.replace("\\", "/")
    path = path.replace("\n", "")
    parts = path.split("/")
    filename = os.path.basename(path)
    filename, extension = os.path.splitext(filename)
    filename_path = "/".join(parts[6:])
    if extension != ".wav":
        return None, None
    audio_name = parts[9] if corpus_name == "nurc_sp" else parts[8]

    if audio_name in acc:
        acc[audio_name]["segments"].append(filename_path)
    else:
        acc[audio_name] = {
            "corpus_name": corpus_name,
            "segments": [filename_path],
        }


for segment_path in nurc_files:
    process_path(audios_on_folders, "nurc_sp", segment_path)

for segment_path in mupe_files:
    process_path(audios_on_folders, "mupe", segment_path)

In [22]:
analyzed_audios = []
if (
    audios_on_db is not None
    and not isinstance(audios_on_db, int)
    and not audios_on_db.empty
):
    for idx, audio in tqdm(audios_on_db.iterrows(), total=audios_on_db.shape[0]):
        audio_id = audio["id"]
        audio_name = audio["name"]
        corpus_name = audio["corpus_id"]
        with Database() as db:
            segments_on_db = get_segments_by_audio_id(audio_id)
        segments_on_db_set = set(segments_on_db["file_path"].to_list())
        found = True
        try:
            audio_on_folder = audios_on_folders[audio_name]
        except KeyError:
            found = False
            name = audio_name.split("_")[-1]
            name = " ".join(name.split(" ")[:2])
            for key in audios_on_folders.keys():
                if name in key:
                    found = True
                    audio_on_folder = audios_on_folders[key]
                    audio_name = key
                    break

        if not found:
            result = {
                "audio_name": audio_name,
                "corpus_name": corpus_name,
                "count_only_on_db": len(segments_on_db_set),
                "only_on_db": {},
                "count_only_on_folders": 0,
                "only_on_folders": {},
            }
            print(result)
            analyzed_audios.append(result)
            print("Audio not found on folders:", audio_name)
            continue

        segments_on_folder_set = set(audios_on_folders[audio_name]["segments"])
        # Unique elements in elements_on_db and not in elements_on_folders
        only_on_db = segments_on_db_set - segments_on_folder_set

        # Unique elements in elements_on_folders and not in elements_on_db
        only_on_folders = segments_on_folder_set - segments_on_db_set

        result = {
            "audio_name": audio_name,
            "corpus_name": corpus_name,
            "count_only_on_db": len(only_on_db),
            "only_on_db": only_on_db,
            "count_only_on_folders": len(only_on_folders),
            "only_on_folders": only_on_folders,
        }
        if len(only_on_db) > 0 or len(only_on_folders) > 0:
            print(result)
        analyzed_audios.append(result)

  3%|▎         | 20/617 [01:01<29:09,  2.93s/it]

{'audio_name': 'SP_EF_087', 'corpus_name': 2, 'count_only_on_db': 0, 'only_on_db': set(), 'count_only_on_folders': 1, 'only_on_folders': {'data/nurc_sp/EF/SP_EF_087/audios/0082_SP_EF_087_704.342_704.402.wav'}}


  4%|▍         | 25/617 [01:15<28:50,  2.92s/it]

{'audio_name': 'SP_EF_353', 'corpus_name': 2, 'count_only_on_db': 0, 'only_on_db': set(), 'count_only_on_folders': 1, 'only_on_folders': {'data/nurc_sp/EF/SP_EF_353/audios/0028_SP_EF_353_344.565_345.165.wav'}}


  5%|▌         | 31/617 [01:33<27:48,  2.85s/it]

...

{'audio_name': '002/CP555', 'corpus_name': 3, 'count_only_on_db': 73, 'only_on_db': {}, 'count_only_on_folders': 0, 'only_on_folders': {}}
Audio not found on folders: 002/CP555


100%|█████████▉| 616/617 [31:26<00:02,  2.81s/it]

{'audio_name': '002/CP554', 'corpus_name': 3, 'count_only_on_db': 78, 'only_on_db': {}, 'count_only_on_folders': 0, 'only_on_folders': {}}
Audio not found on folders: 002/CP554


100%|██████████| 617/617 [31:29<00:00,  3.06s/it]

{'audio_name': '002/CP553', 'corpus_name': 3, 'count_only_on_db': 104, 'only_on_db': {}, 'count_only_on_folders': 0, 'only_on_folders': {}}
Audio not found on folders: 002/CP553


In [23]:
df = pd.DataFrame(analyzed_audios)

In [24]:
df[(df.count_only_on_db > 0) | (df.count_only_on_folders > 0)]

,audio_name,corpus_name,count_only_on_db,only_on_db,count_only_on_folders,only_on_folders
19,SP_EF_087,2,0,{},1,{data/nurc_sp/EF/SP_EF_087/audios/0082_SP_EF_0...
24,SP_EF_353,2,0,{},1,{data/nurc_sp/EF/SP_EF_353/audios/0028_SP_EF_3...
30,SP_EF_366,2,0,{},1,{data/nurc_sp/EF/SP_EF_366/audios/0286_SP_EF_3...
34,SP_EF_385,2,0,{},1,{data/nurc_sp/EF/SP_EF_385/audios/0254_SP_EF_3...
41,SP_EF_387,2,0,{},2,{data/nurc_sp/EF/SP_EF_387/audios/0231_SP_EF_3...
...,...,...,...,...,...,...
612,SP_EF_187,3,70,{data/CertasPalavras/002/CP557_1/audios/0012_C...,242,{data/nurc_sp/EF/SP_EF_187/audios/0217_SP_EF_1...
613,002/CP556,3,99,{},0,{}
614,002/CP555,3,73,{},0,{}
615,002/CP554,3,78,{},0,{}


In [26]:
df.to_excel("audios_consistency.xlsx", index=False)

### 3. Stuck on annotator who left

In [27]:
query = """
SELECT * 
FROM Audio 
LEFT JOIN users on Audio.responsible_user_id = users.id 
WHERE 
    (Audio.finished = 0 OR Audio.finished is null)
    AND (Audio.error_flag = 0 OR Audio.error_flag is null)
		AND Audio.responsible_user_id is not null
"""

with Database() as db:
    audios_with_user = db._run_query(query)

In [30]:
audios_with_user["data_fim"] = pd.to_datetime(
    audios_with_user["data_fim"], format="%d/%m/%Y"
)
audios_with_user["data_inicio"] = pd.to_datetime(
    audios_with_user["data_inicio"], format="%d/%m/%Y"
)

In [31]:
# get all audios where user end date is before today
audios_with_user[audios_with_user["data_fim"] < pd.Timestamp.today()]

,id,created_at,name,num_speakers,json_metadata,duration,corpus_id,finished,responsible_user_id,to_review,...,id,username,password,salt,last_login_time,last_login_ip,carga_horaria,data_inicio,data_fim,designated_corpus_id
11,461,2023-09-08 04:45:54,PC_MA_HV249_Horacio_Calligaris_Galvanese,None,"{""race"": null, ""title"": ""Pelo exercício da cid...",10099.4,1,None,130,None,...,130,marinaaluisio@yahoo.com.br,d5f6c8e69c7367d7ed59b53123ed7e721262290d1442ea...,NmRr1wJ6,2023-11-26 14:40:33,177.106.42.175,6,2023-09-01,2023-09-29,2


### 4. Fix offsets